In [ ]:
from google.colab import userdata
from huggingface_hub import login
from transformers import AutoTokenizer,AutoModelForCausalLM,BitsAndBytesConfig,TextStreamer


In [ ]:
!pip install accelerate

In [1]:
hf_token=userdata.get('HF_TOKEN')
login(hf_token)

NameError: name 'userdata' is not defined

In [ ]:
tokenizer=AutoTokenizer.from_pretrained("google/gemma-7b", trust_remote_code=True)
input_ids = tokenizer.encode("what is best model for classification. where should i get it from \
  how to evaluate it and know it works good and have tokenizer")
print(input_ids)

print(tokenizer.decode(input_ids))
print(tokenizer.batch_decode(input_ids))

[2, 5049, 603, 1963, 2091, 604, 17283, 235265, 1570, 1412, 496, 947, 665, 774, 140, 1139, 577, 18739, 665, 578, 1230, 665, 3598, 1426, 578, 791, 142224]
<bos>what is best model for classification. where should i get it from   how to evaluate it and know it works good and have tokenizer
['<bos>', 'what', ' is', ' best', ' model', ' for', ' classification', '.', ' where', ' should', ' i', ' get', ' it', ' from', '   ', 'how', ' to', ' evaluate', ' it', ' and', ' know', ' it', ' works', ' good', ' and', ' have', ' tokenizer']


In [ ]:
llama_tokenizer=AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B-Instruct", trust_remote_code=True)
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "what is best model for classification. where should i get it from \
  how to evaluate it and know it works good and have tokenizer"},
]
prompt=llama_tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print(prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a helpful assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>

what is best model for classification. where should i get it from   how to evaluate it and know it works good and have tokenizer<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [ ]:
phi_tokenizer=AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "what is best model for classification. where should i get it from \
  how to evaluate it and know it works good and have tokenizer"},
]
prompt=phi_tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print(prompt)

text="Hello how are you?"
tokens=phi_tokenizer.tokenize(text)
print(tokens)
print(phi_tokenizer.convert_tokens_to_ids(tokens))

<|system|>
You are a helpful assistant.<|end|>
<|user|>
what is best model for classification. where should i get it from   how to evaluate it and know it works good and have tokenizer<|end|>
<|assistant|>

['▁Hello', '▁how', '▁are', '▁you', '?']
[15043, 920, 526, 366, 29973]


In [ ]:
#instruct models
LLAMA="meta-llama/Meta-Llama-3.1-8B-Instruct"
PHI="microsoft/Phi-3.5-mini-instruct"
GEMMA="google/gemma-2-2b-it"
QWEN="Qwen/Qwen2.5-7B-Instruct"
MIXTRAL="mistralai/Mixtral-8x7B-Instruct-v0.1"


messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Tell a light hearted joke to a room full of scientists "},
]

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)




In [ ]:
llama_tokenizer=AutoTokenizer.from_pretrained(LLAMA, trust_remote_code=True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
inputs=llama_tokenizer.apply_chat_template(messages,return_tensors="pt",return_dict=True).to("cuda")
model=AutoModelForCausalLM.from_pretrained(LLAMA, quantization_config=quant_config, device_map="auto", trust_remote_code=True)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
inputs=llama_tokenizer.apply_chat_template(messages,return_tensors="pt",return_dict=True).to("cuda")
print(inputs)

NameError: name 'llama_tokenizer' is not defined

In [ ]:
memory=model.get_memory_footprint()
print(memory)

5591539968


In [ ]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((409

In [ ]:
print(inputs)
outputs=model.generate(inputs=inputs["input_ids"],max_new_tokens=100)
print(llama_tokenizer.decode(outputs[0],skip_special_tokens=True))

NameError: name 'inputs' is not defined

In [ ]:
import torch
#clean up
del inputs , outputs, model, llama_tokenizer
torch.cuda.empty_cache()

NameError: name 'inputs' is not defined

In [ ]:
!nvidia-smi


Sat Jan 18 02:26:40 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P0              30W /  70W |  10285MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import torch
def generate(model,message):
  tokenizer=AutoTokenizer.from_pretrained(model, trust_remote_code=True)
  tokenizer.pad_token = tokenizer.eos_token
  stream=TextStreamer(tokenizer)
  inputs=tokenizer.apply_chat_template(messages,return_tensors="pt",return_dict=True).to("cuda")
  model=AutoModelForCausalLM.from_pretrained(model, quantization_config=quant_config, device_map="auto")
  outputs=model.generate(inputs=inputs["input_ids"],max_new_tokens=100,streamer=stream)
  #clean up
  del tokenizer,model ,stream,inputs , outputs
  torch.cuda.empty_cache()




In [ ]:
generate(LLAMA,messages)

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a helpful assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>

Tell a light hearted joke to a room full of 

/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


scientists<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Here's a joke that combines science and humor:

Why did the physicist break up with his girlfriend?

Because he found her mass attractive, but her charge was always negative!

(pause for laughter)

I hope that one bonded with you all! (get it? bonded... like a chemical reaction... ahh, science puns)<|eot_id|>


In [ ]:
generate(QWEN,messages)

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Tell a light hearted joke to a room full of scientists <|im_end|>
<|im_start|>-Type of scientist: Astrophysicist
<|im_start|>-Style of joke: Playful pun
<|im_start|>
<|im_start|>
Why did the astrophysicist refuse to go to the beach? Because he didn't want to deal with solar masses of sand!<|im_end|>


In [ ]:
!pip install -U accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.6/336.6 kB 8.9 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.2.1
    Uninstalling accelerate-1.2.1:
      Successfully uninstalled accelerate-1.2.1
